# 0. Setup: Generación de Archivos de Prueba

Ejecuta esto una sola vez para crear los archivos .csv y .parquet con los que practicaremos.

In [1]:
import pandas as pd 
import numpy as np
import time
import os 

In [2]:
#1. Creamos un DataFrame grande(10,000 filas)
df_dummy = pd.DataFrame({
    'fecha': pd.date_range(start='2020-01-01', periods=10000,freq='h'),
    'producto': np.random.choice(['Laptop','Mouse','Teclado','Monitor'], 10000),
    'precio': np.random.randint(20,2000,10000),
    'cliente_id':np.random.randint(1000, 9999, 10000),
    'comentarios': ['Texto largo de relleno para ocupar memoria RAM innecesariamente'] * 10000
})

#2. Guardamos en formato CSV(Texto Lento)
df_dummy.to_csv('Archivos/ventas_grandes.csv', index=False)

#3. Guardamos en formato Parquet(Binario rápido)
# Nota: Si te da error, necesitas instalar: pip install pyarrow
try:
    df_dummy.to_parquet('Archivos/ventas_grande.parquet')
    print("Archivos 'ventas_grande.csv' y 'ventas_grande.parquet' creados.")
except ImportError:
    print("No se pudo crear Parquet. Instala pyarrow. Solo usaremos CSV.")

Archivos 'ventas_grande.csv' y 'ventas_grande.parquet' creados.


# Ejemplo 1: Estrategia de Selección (usecols)

Teoría: No cargues basura. Si solo necesitas analizar precios por producto, ¿para qué cargar los comentarios largos?

In [3]:
print("--- CARGA INGENUA (Carga todo) ---")
df_full=pd.read_csv('Archivos/ventas_grandes.csv')
print(f"Memoria Full:{df_full.memory_usage(deep=True).sum()} bytes")

print("\n--- CARGA ESTRATÉGICA (Solo columnas útiles) ---")
#usecols: Solo traemos lo que necesitamos
df_opt=pd.read_csv('Archivos/ventas_grandes.csv', usecols=['producto','precio'])
print(f"Memoria Optimizada:{df_opt.memory_usage(deep=True).sum()} bytes")

#Calculo de ahorro
ahorro=1-(df_opt.memory_usage(deep=True).sum()/df_full.memory_usage(deep=True).sum())
print(f"Ahorro de Memoria:{ahorro:.2%}")

--- CARGA INGENUA (Carga todo) ---
Memoria Full:1282679 bytes

--- CARGA ESTRATÉGICA (Solo columnas útiles) ---
Memoria Optimizada:222679 bytes
Ahorro de Memoria:82.64%


Explicación: Al ignorar la columna 'comentarios', la carga es instantánea y ocupa una fracción de la RAM. Esta es la técnica #1 de Matt Harrison.

# Ejemplo 2: Optimización de Tipos (dtype y parse_dates)

Teoría: Evitar que Pandas "adivine" los tipos. Le decimos explícitamente qué es cada cosa para comprimir memoria al vuelo.

In [4]:
print("--- CARGA INTELIGENTE (Tipos definidos) ---")

#Definimos la estrategia
tipos_optimizados = {
    'producto': 'category', #De object a category (Ahorro Masivo)
    'precio': 'int16'   #De int64 a int16 (Ahorro de Bits)
}

df_smart=pd.read_csv('Archivos/ventas_grandes.csv',
                    usecols=['fecha','producto','precio'],  #Filtrado vertical
                    dtype=tipos_optimizados,    #compresión de tipos
                    parse_dates=['fecha'],  #Parseo temporal inmediato
                    index_col='fecha'   #Indexación automática
)

print(df_smart.info())
print("\nObserva: El índice ya es Datetime y 'producto' es Category.")


--- CARGA INTELIGENTE (Tipos definidos) ---
<class 'pandas.DataFrame'>
DatetimeIndex: 10000 entries, 2020-01-01 00:00:00 to 2021-02-20 15:00:00
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   producto  10000 non-null  category
 1   precio    10000 non-null  int16   
dtypes: category(1), int16(1)
memory usage: 107.6 KB
None

Observa: El índice ya es Datetime y 'producto' es Category.


Explicación: Hemos combinado 4 pasos en 1. El DataFrame nace limpio, indexado y optimizado. No hace falta hacer limpieza posterior (astype) ni set_index.

In [5]:
print("--- CARGA POR LOTES (Chunking) ---")

#Supongamos que 10000 filas es "demasiado" para mi RAM
#Cargaremos de 2000 en 2000

chunck_iterator=pd.read_csv('Archivos/ventas_grandes.csv',chunksize=2000)

print(f"Objeto recibido:{type(chunck_iterator)}") #No es un DataFrame, es un TextFileReader
total_ventas=0

#Iteramos como si fuera una lista
for i,lote_df in enumerate(chunck_iterator):
    # En cada vuelta, 'lote_df' es un pequeño DataFrame de 2000 filas
    suma_lote=lote_df['precio'].sum()
    total_ventas+=suma_lote
    print(f"Lote {i+1}: Procesadas {len(lote_df)}filas. Suma Parcial:{suma_lote}")

print(f"\nVenta Total Calculada (Sin cargar todo a la vez): {total_ventas}")

--- CARGA POR LOTES (Chunking) ---
Objeto recibido:<class 'pandas.io.parsers.readers.TextFileReader'>


Lote 1: Procesadas 2000filas. Suma Parcial:2003543
Lote 2: Procesadas 2000filas. Suma Parcial:1979053
Lote 3: Procesadas 2000filas. Suma Parcial:1991216
Lote 4: Procesadas 2000filas. Suma Parcial:1991630
Lote 5: Procesadas 2000filas. Suma Parcial:2020899

Venta Total Calculada (Sin cargar todo a la vez): 9986341


Explicación: Esta es la única forma de procesar archivos de Gigabytes en una laptop normal. Nunca cargas todo el archivo, solo fragmentos.

# La Velocidad Binaria (read_parquet)

Teoría: Leer texto (CSV) es lento porque la CPU tiene que leer letra por letra. Parquet es binario, la CPU solo copia memoria.

In [6]:
print("--- DUELO DE VELOCIDAD: CSV vs PARQUET ---")

#1. Medir CSV
inicio=time.time()
df_csv=pd.read_csv('Archivos/ventas_grandes.csv')
fin=time.time()
tiempo_csv=fin-inicio
print(f"Tiempo CSV: {tiempo_csv:.4f} segundos")

#2. Medir Parquet (Si tienes pyrrow instalado)
try:
    inicio=time.time()
    df_parquet=pd.read_parquet("Archivos/ventas_grande.parquet")
    fin=time.time()
    tiempo_parquet=fin-inicio
    print(f"Tiempo Parquet: {tiempo_parquet:.4f} segundos")

    print(f"Parquet es {tiempo_csv / tiempo_parquet:.1f} veces más rápido.")
except:
    print("Salto paso Parquet (falta librería).")

--- DUELO DE VELOCIDAD: CSV vs PARQUET ---
Tiempo CSV: 0.0384 segundos
Tiempo Parquet: 0.0847 segundos
Parquet es 0.5 veces más rápido.


Explicación: En archivos grandes reales, Parquet suele ser entre 10 y 50 veces más rápido que CSV y ocupa la mitad de espacio en disco. Es el formato profesional por excelencia.